In [ ]:
%load_ext autoreload
%autoreload 2

In [21]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
from mesh_import import mesh
import mesh.affix as affix
from mesh.affix import CkipAffixoids
from mesh.affix import AffixoidCtmDataset, ByCharCtmProcessor

In [3]:
affixoids = CkipAffixoids()

In [14]:
asbc_dir = mesh.get_data_dir() / "asbc/"
with (asbc_dir/"asbc5_characters.pkl").open("rb") as fin:
    as_cfreq = pickle.load(fin)
with (asbc_dir/"asbc5_words.pkl").open("rb") as fin:
    as_wfreq = pickle.load(fin)

In [6]:
aff_chars = set(x.affixoid for x in affixoids)

In [7]:
len(aff_chars)

2471

In [33]:
from collections import defaultdict
charpos = defaultdict(lambda: {"prefix":[], "suffix": []})
for w, f in tqdm(as_wfreq.items()):
    if not w:
        print(f"empty word??{w}")
        continue
    if len(w) == 1:
        continue
    cstart = w[0]
    cend = w[-1]
    cstart2 = w[:2]
    cend2 = w[-2:]
    w_tuple = (w,f)
    if cstart in aff_chars:
        charpos[cstart]["prefix"].append(w_tuple)
    if cstart2 in aff_chars:
        charpos[cstart2]["prefix"].append(w_tuple)
    if cend in aff_chars:
        charpos[cend]["suffix"].append(w_tuple)    
    if cend2 in aff_chars:
        charpos[cend2]["suffix"].append(w_tuple)    

empty word??



In [45]:
from numpy.random import multinomial
from mesh.senses.corpus_streamer import CorpusStreamer
from itertools import islice, cycle

In [44]:
asbc = CorpusStreamer()

loading ASBC corpus streamer index


In [42]:
fvec = [1,2,3]
pvec = fvec/np.sum(fvec)
multinomial(1200, pvec)

array([209, 387, 604])

In [51]:
def sample_sentences(word_list):
    freq = [x[1] for x in word_list]
    pvec = freq/np.sum(freq)
    sample_vec = np.random.multinomial(100, pvec)
    sentences = []
    for idx, n_sent in enumerate(sample_vec):
        word = word_list[idx][0]
        sents = islice(asbc.query(word), n_sent)
        sentences.append(list(zip(cycle([word]), sents)))
    return sentences

sample_sentences(charpos["走"]["prefix"])

[[('走出',
   [('有時', 'D'),
    ('明明', 'D'),
    ('走出', 'VCL'),
    ('了', 'Di'),
    ('一', 'Neu'),
    ('大', 'VH'),
    ('步', 'Nf'),
    ('，', 'COMMACATEGORY')]),
  ('走出',
   [('走出', 'VCL'),
    ('這', 'Nep'),
    ('個', 'Nf'),
    ('人生', 'Na'),
    ('。', 'PERIODCATEGORY')]),
  ('走出',
   [('中研院', 'Nc'),
    ('應該', 'D'),
    ('能', 'D'),
    ('走出', 'VCL'),
    ('一', 'Neu'),
    ('條', 'Nf'),
    ('康莊大道', 'Na'),
    ('。', 'PERIODCATEGORY')]),
  ('走出',
   [('是否', 'D'),
    ('真', 'D'),
    ('能', 'D'),
    ('促使', 'VL'),
    ('日本', 'Nc'),
    ('走出', 'VCL'),
    ('二', 'Neu'),
    ('次', 'Nf'),
    ('戰後', 'Nd'),
    ('以來', 'Ng'),
    ('最', 'Dfa'),
    ('嚴重', 'VH'),
    ('的', 'DE'),
    ('經濟', 'Na'),
    ('衰退', 'VH'),
    ('陰影', 'Na'),
    ('？', 'QUESTIONCATEGORY')]),
  ('走出',
   [('才', 'Da'),
    ('能', 'D'),
    ('走出', 'VCL'),
    ('新意', 'Na'),
    ('，', 'COMMACATEGORY')]),
  ('走出',
   [('金馬獎', 'Nb'), ('走出', 'VCL'), ('台北市', 'Nc'), ('，', 'COMMACATEGORY')])],
 [('走過', [('提前', 'VB'), ('走過', 'VCL'), ('適應